[View in Colaboratory](https://colab.research.google.com/github/meghamattikalli/lstm/blob/master/lstm_95_new.ipynb)

In [1]:
import os

from google.colab import files
uploaded = files.upload()

Saving five_thousand.csv to five_thousand.csv
Saving Indian-Female-Names1.csv to Indian-Female-Names1.csv
Saving Indian-Male-Names1.csv to Indian-Male-Names1.csv


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 29 00:07:33 2018
acc= 95.52

@author: HP
"""


from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np


import io

female = pd.read_csv(io.StringIO(uploaded['Indian-Female-Names1.csv'].decode('utf-8')))

male = pd.read_csv(io.StringIO(uploaded['Indian-Male-Names1.csv'].decode('utf-8')))

eng_word = pd.read_csv(io.StringIO(uploaded['five_thousand.csv'].decode('utf-8')))


#parameters
maxlen = 10
labels = 2

# Importing the dataset
#female= pd.read_csv('Indian-Female-Names1.csv')
#male= pd.read_csv('Indian-Male-Names1.csv')
names=pd.concat([female,male], axis=0)
names= names.apply(lambda x: x.astype(str).str.lower())
names=names.iloc[:,0:1]
names["type"]=1
names=names[names['word'].map(len) > 2]

unwanted=['mr','.','moh.','@','miss','&','mrs','/','ku.','km',',','km0','-','(',')','smt','smt.','`','[','ms','1','2','3','4','5','6','7','8','9','0']
for i in unwanted:
    names['word'] = names.word.str.replace(i, '')
names['word'] = names.word.str.strip()
names=names.drop_duplicates(subset=['word'], keep='first')
names['word'] = names.word.str.partition(" ")


#english words
#eng_word = pd.read_csv('five_thousand.csv')
eng_word = eng_word.apply(lambda x: x.astype(str).str.lower())
eng_word['word'] = eng_word.word.str.replace('-', '')
eng_word['word'] = eng_word.word.str.replace("'", '')
eng_word['word'] = eng_word.word.str.replace("/", ' ')
eng_word['word'] = eng_word.word.str.partition(" ")


#eng_pre = pd.read_csv('prepositions.csv')
eng_word['type'] = 0
#eng_pre['type'] = 0

#concatenate names and english words
dataset=pd.concat([names,eng_word], axis=0)
dataset=dataset.dropna()

dataset.index=[i for i in range(0,len(dataset.index))] #renaming the indices
features = dataset['word']
label = dataset['type']

vocab = set(' '.join([str(i) for i in features]))

vocab.add('END')
len_vocab = len(vocab)

print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(dataset))

char_index = dict((c, i) for i, c in enumerate(vocab))

print(char_index)


#train test split
msk = np.random.rand(len(dataset)) < 0.8
train = dataset[msk]
test = dataset[~msk]


def set_flag(i):
    tmp = np.zeros(28);
    tmp[i] = 1
    return(tmp)
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.word]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.type:
    if i == 0:
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])

np.asarray(train_X).shape

np.asarray(train_Y).shape

"""#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)"""

#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.word]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.type:
    if i == 0:
        test_Y.append([1,0]) 
    else:
        test_Y.append([0,1])
        

print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

train_X=np.array(train_X)
train_Y=np.array(train_Y)
test_Y=np.array(test_Y)
test_X=np.array(test_X)


batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=50,validation_data=(test_X, test_Y))

score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)




Using TensorFlow backend.


{'l', 'a', 'e', 'x', 'f', 'd', 'j', 'z', ' ', 'END', 'q', 't', 'w', 'p', 'b', 'r', 's', 'n', 'o', 'g', 'v', 'c', 'i', 'm', 'y', 'k', 'u', 'h'}
vocab length is  28
length of input is  18716
{'l': 0, 'a': 1, 'e': 2, 'x': 3, 'f': 4, 'd': 5, 'j': 6, 'z': 7, ' ': 8, 'END': 9, 'q': 10, 't': 11, 'w': 12, 'p': 13, 'b': 14, 'r': 15, 's': 16, 'n': 17, 'o': 18, 'g': 19, 'v': 20, 'c': 21, 'i': 22, 'm': 23, 'y': 24, 'k': 25, 'u': 26, 'h': 27}
Build model...
(3775, 10, 28)
(3775, 2)


/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 14941 samples, validate on 3775 samples
Epoch 1/50
14941/14941 [==============================] - 72s 5ms/step - loss: 0.4829 - acc: 0.7811 - val_loss: 0.3699 - val_acc: 0.8442
Epoch 2/50
14941/14941 [==============================] - 69s 5ms/step - loss: 0.3506 - acc: 0.8468 - val_loss: 0.3301 - val_acc: 0.8585
Epoch 3/50
14941/14941 [==============================] - 70s 5ms/step - loss: 0.3200 - acc: 0.8580 - val_loss: 0.3007 - val_acc: 0.8736
Epoch 4/50
14941/14941 [==============================] - 70s 5ms/step - loss: 0.2963 - acc: 0.8755 - val_loss: 0.2772 - val_acc: 0.8866
Epoch 5/50
14941/14941 [==============================] - 70s 5ms/step - loss: 0.2837 - acc: 0.8845 - val_loss: 0.2705 - val_acc: 0.8885
Epoch 6/50
14941/14941 [==============================] - 71s 5ms/step - loss: 0.2713 - acc: 0.8907 - val_loss: 0.2494 - val_acc: 0.8999
Epoch 7/50
14941/14941 [==============================] - 72s 5ms/step - loss: 0.2587 - acc: 0.8941 - val_loss: 0.2474 - val_acc:

In [0]:
# Predicting the Test set results
y_pred = model.predict(test_X)
y_pred = (y_pred > 0.5)





In [0]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_Y.argmax(axis=1), y_pred.argmax(axis=1))
#cm = confusion_matrix(test_Y, y_pred)

In [10]:
cm

array([[ 905,  106],
       [  63, 2701]])

In [11]:

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
# Print model performance and plot the roc curve
print('accuracy is {:.3f}'.format(accuracy_score(test_Y,y_pred)))
print('roc-auc is {:.3f}'.format(roc_auc_score(test_Y,y_pred)))

accuracy is 0.955
roc-auc is 0.936
